In [1]:
import pandas as pd

In [2]:
import csv

In [16]:
import json

In [32]:
import uuid

In [3]:
import boto3

In [144]:
from hashlib import sha256

In [11]:
df = pd.read_csv("scrubbed.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
pd.set_option('display.max_columns', None)

In [27]:
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')

In [28]:
df

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611
...,...,...,...,...,...,...,...,...,...,...,...
80327,9/9/2013 21:15,nashville,tn,us,light,600,10 minutes,Round from the distance/slowly changing colors...,9/30/2013,36.165833,-86.784444
80328,9/9/2013 22:00,boise,id,us,circle,1200,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,9/30/2013,43.613611,-116.202500
80329,9/9/2013 22:00,napa,ca,us,other,1200,hour,Napa UFO&#44,9/30/2013,38.297222,-122.284444
80330,9/9/2013 22:20,vienna,va,us,circle,5,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.901111,-77.265556


In [90]:
df.dtypes

datetime                 object
city                     object
state                    object
country                  object
shape                    object
duration (seconds)       object
duration (hours/min)     object
comments                 object
date posted              object
latitude                float64
longitude               float64
dtype: object

In [154]:
json_list = json.loads(df.to_json(orient='records'))

In [155]:
json_list[0]

{'datetime': '10/10/1949 20:30',
 'city': 'san marcos',
 'state': 'tx',
 'country': 'us',
 'shape': 'cylinder',
 'duration (seconds)': '2700',
 'duration (hours/min)': '45 minutes',
 'comments': 'This event took place in early fall around 1949-50. It occurred after a Boy Scout meeting in the Baptist Church. The Baptist Church sit',
 'date posted': '4/27/2004',
 'latitude': 29.8830556,
 'longitude ': -97.9411111}

In [173]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('article_ufo_sightings')

In [157]:
with table.batch_writer() as batch:
    for record in json_list:
        if record['longitude '] and record['latitude']:
            record['id'] = sha256(str(record).encode()).hexdigest()
            try:
                record['latitude'] = int(record['latitude'])
                record['longitude'] = int(record['longitude '])
                try:
                    record['duration'] = int(float(record['duration (seconds)']))
                except:
                    pass
                record.pop('longitude ')
                record.pop('date posted')
                record.pop('duration (hours/min)')
                record.pop('duration (seconds)')
                batch.put_item(Item=record)
            except Exception as e:
                print(record)
                raise e 

In [158]:
dfb = pd.read_csv("GlobalLandTemperaturesByMajorCity.csv")

In [159]:
dfb

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1849-01-01,26.704,1.435,Abidjan,Côte D'Ivoire,5.63N,3.23W
1,1849-02-01,27.434,1.362,Abidjan,Côte D'Ivoire,5.63N,3.23W
2,1849-03-01,28.101,1.612,Abidjan,Côte D'Ivoire,5.63N,3.23W
3,1849-04-01,26.140,1.387,Abidjan,Côte D'Ivoire,5.63N,3.23W
4,1849-05-01,25.427,1.200,Abidjan,Côte D'Ivoire,5.63N,3.23W
...,...,...,...,...,...,...,...
239172,2013-05-01,18.979,0.807,Xian,China,34.56N,108.97E
239173,2013-06-01,23.522,0.647,Xian,China,34.56N,108.97E
239174,2013-07-01,25.251,1.042,Xian,China,34.56N,108.97E
239175,2013-08-01,24.528,0.840,Xian,China,34.56N,108.97E


In [164]:
dfb[dfb['Country'] == 'United States'].City.value_counts()

New York       3239
Chicago        3239
Los Angeles    1977
Name: City, dtype: int64

In [165]:
dfb.dtypes

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                              object
Country                           object
Latitude                          object
Longitude                         object
dtype: object

In [195]:
json_list_cities = json.loads(dfb.to_json(orient='records'))

In [196]:
json_list_cities[0]

{'dt': '1849-01-01',
 'AverageTemperature': 26.704,
 'AverageTemperatureUncertainty': 1.435,
 'City': 'Abidjan',
 'Country': "Côte D'Ivoire",
 'Latitude': '5.63N',
 'Longitude': '3.23W'}

In [197]:
table_climate = dynamodb.Table('article_ufo_climate')

In [198]:
with table_climate.batch_writer() as batch:
    for record in json_list_cities:
        try:
            record['AverageTemperature'] = int(record['AverageTemperature'])
        except Exception as e :
            pass
        record.pop('AverageTemperatureUncertainty')
        record['Latitude'] = int(float(record['Latitude'][:-1])) * (1 if record['Latitude'][-1] == 'N' else -1)
        record['Longitude'] = int(float(record['Longitude'][:-1])) * (1 if record['Longitude'][-1] == 'E' else -1)
        record['id'] = sha256(str(record).encode()).hexdigest()
        batch.put_item(Item=record)

In [199]:
json_list_cities[0]

{'dt': '1849-01-01',
 'AverageTemperature': 26,
 'City': 'Abidjan',
 'Country': "Côte D'Ivoire",
 'Latitude': 5,
 'Longitude': -3,
 'id': 'd3737dbd474f374f98b3d31d891b11f3e203bc6662861edb402f5e44f9af9da6'}